In [ ]:
# Welcome to the CBC MILP Solver 
# Version: Trunk
# Build Date: Oct 28 2021 

# Starting solution of the Linear programming relaxation problem using Primal Simplex

# The problem does not have an optimal solution.


In [ ]:
# from ortools.linear_solver import pywraplp
# The problem does not have an optimal solution.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('usa_men.csv')


In [3]:
dfs = pd.read_csv('../../Data/cleandata22-23/usa_men22_23.csv')

In [4]:
dfs

,Name,Apparatus,D_Score,E_Score,Penalty,Score
0,adriel perales-valencia,PB,5.200,8.7000,0.000,13.9000
1,adriel perales-valencia,SR,5.400,7.4500,0.000,12.8500
2,aidan cuy,FX,4.700,8.2000,0.000,12.9000
3,aidan cuy,HB,4.000,7.6500,0.000,11.6500
4,aidan cuy,PB,4.800,8.1500,0.000,12.9500
...,...,...,...,...,...,...
770,zachary nunez,HB,4.500,8.6500,0.000,13.1500
771,zachary nunez,PB,5.125,8.1125,0.000,13.2375
772,zachary nunez,PH,5.650,7.6250,0.000,13.2750
773,zachary nunez,SR,4.700,8.2500,0.000,12.9500


In [ ]:
df2=df.copy()

In [ ]:
# Asher Hong
# Fred Richard
# Khoi Young
# Vitaliy Guimaraes
# Yul Moldauer

In [ ]:
selected_athletes = ['asher hong', 'fred richard', 'khoi young', 
                     'vitaliy guimaraes', 'yul moldauer']

# Filter DataFrame based on the list of athletes
filtered_df = df[df['Athlete'].isin(selected_athletes)]



In [ ]:
df = filtered_df.copy()

In [ ]:
df.iloc[4:5]

In [ ]:
df

In [ ]:
sorted_df = df.sort_values(by='AA', ascending=False)

In [ ]:
df=sorted_df.copy()

In [ ]:
df = df.iloc[0:5]

In [ ]:
df=df.drop(columns=['Unnamed: 0', 'Rank'])

In [ ]:
df= df.drop(columns=['AA'])

In [ ]:
import pandas as pd
from itertools import combinations
import pandas as pd
from pyqubo import Array, Constraint, Placeholder, solve_qubo
from itertools import combinations

In [ ]:
df

In [ ]:
# import pandas as pd
# import networkx as nx
# from networkx.algorithms import bipartite



# # Create a bipartite graph
# B = nx.Graph()
# B.add_nodes_from(df['Athlete'], bipartite=0)
# B.add_nodes_from(df.columns[1:-1], bipartite=1)

# # Add weighted edges
# for i, row in df.iterrows():
#     for j in range(1, len(df.columns)-1):
#         B.add_edge(row['Athlete'], df.columns[j], weight=row[df.columns[j]])

# # Solve the weighted bipartite matching problem
# matching = bipartite.maximum_matching(B)

# # Extract the selected athletes and their corresponding apparatus
# selected_athletes = {athlete: apparatus for athlete, apparatus in matching.items() if isinstance(apparatus, str)}

# # Print the selected athletes and their corresponding apparatus
# for athlete, apparatus in selected_athletes.items():
#     print(f"{athlete} will compete on {apparatus}")


In [ ]:
# data = {
#     'Athlete': ['khoi young', 'fred richard', 'yul moldauer', 'vitaliy guimaraes', 
#                 'asher hong','vitaliy guimaraes', 'asher hong', 'fred richard',
#                 'yul moldauer', 'khoi young'],
#     'Apparatus': ['FX', 'PH', 'SR', 'VT', 'PB', 'VT', 'PB', 'PH', 'SR', 'FX' ]
# }

# df = pd.DataFrame(data)

In [ ]:
# df['Apparatus'].value_counts()

In [ ]:
# df['Athlete'].value_counts()

In [ ]:
df

In [ ]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize

# Sample DataFrame
data = {
    'Athlete': ['yul moldauer', 'fred richard', 'vitaliy guimaraes', 'khoi young', 'asher hong'],
    'FX': [14.400, 14.483, 14.250, 13.550, 14.029],
    'PH': [13.957, 13.357, 14.350, 15.342, 12.949],
    'SR': [14.114, 13.350, 13.600, 12.550, 14.566],
    'VT': [14.250, 14.000, 14.450, 15.244, 16.680],
    'PB': [15.121, 14.450, 13.900, 13.750, 14.355],
    'HB': [13.500, 14.962, 13.400, 13.100, 10.450],
}

df = pd.DataFrame(data)

# Number of athletes and apparatus
num_athletes = len(df)
num_apparatus = len(df.columns) - 1  # Exclude 'Athlete' column

# Binary decision variables
athlete_vars = {athlete: LpVariable(f'athlete_{athlete}', cat='Binary') for athlete in df['Athlete']}
apparatus_vars = {apparatus: LpVariable(f'apparatus_{apparatus}', cat='Binary') for apparatus in df.columns[1:]}

# Continuous decision variables for scores
score_vars = {(athlete, apparatus): LpVariable(f'score_{athlete}_{apparatus}', cat='Binary')
              for athlete in df['Athlete'] for apparatus in df.columns[1:]}

# Create the optimization problem
prob = LpProblem("Athlete_Apparatus_Assignment", LpMaximize)

# Objective function: maximize the total AA score
prob += lpSum(df.loc[i, apparatus] * score_vars[df['Athlete'][i], apparatus]
              for i in range(num_athletes) for apparatus in df.columns[1:]), "Total_Score"

# Constraints: each athlete competes on each apparatus exactly 3 times
for i in range(num_athletes):
    for apparatus in df.columns[1:]:
        prob += lpSum(score_vars[df['Athlete'][i], app] for app in df.columns[1:]) == 3 * athlete_vars[df['Athlete'][i]]

# Constraints: each apparatus is competed on exactly 3 times
for apparatus in df.columns[1:]:
    prob += lpSum(athlete_vars[athlete] for athlete in df['Athlete']) == 4, f'Apparatus_{apparatus}_constraint'

# Solve the problem
prob.solve()

# Extract the selected athletes and their corresponding apparatus
selected_athletes = [athlete for athlete, var in athlete_vars.items() if var.value() == 1]
selected_apparatus = [apparatus for apparatus, var in apparatus_vars.items() if var.value() == 1]



    
# Print the selected athletes and their corresponding apparatus
for athlete in selected_athletes:
    athlete_apparatus = [apparatus for apparatus in df.columns[1:] if score_vars[athlete, apparatus].value() == 1]
    print(f"{athlete} ")


In [ ]:
df

In [ ]:
df2=df2.drop(columns=['Unnamed: 0', 'Rank'])

In [ ]:
df_sorted = df2.sort_values(by='AA', ascending=False)

In [ ]:
df_sorted

In [ ]:
df_unique = df_sorted.drop_duplicates(subset='Athlete', keep='first')

In [ ]:
dfs = df_unique.copy()

In [ ]:
dfs=dfs.drop(columns=['AA'])

In [ ]:
dfs

In [5]:
from pulp import LpProblem, LpVariable, lpSum, value, LpMaximize

In [6]:
dfs = pd.read_csv('../../Data/cleandata22-23/usa_men22_23.csv')

In [7]:
df=dfs.copy()

In [8]:
df


,Name,Apparatus,D_Score,E_Score,Penalty,Score
0,adriel perales-valencia,PB,5.200,8.7000,0.000,13.9000
1,adriel perales-valencia,SR,5.400,7.4500,0.000,12.8500
2,aidan cuy,FX,4.700,8.2000,0.000,12.9000
3,aidan cuy,HB,4.000,7.6500,0.000,11.6500
4,aidan cuy,PB,4.800,8.1500,0.000,12.9500
...,...,...,...,...,...,...
770,zachary nunez,HB,4.500,8.6500,0.000,13.1500
771,zachary nunez,PB,5.125,8.1125,0.000,13.2375
772,zachary nunez,PH,5.650,7.6250,0.000,13.2750
773,zachary nunez,SR,4.700,8.2500,0.000,12.9500


In [10]:
average_scores = df.groupby(['Name', 'Apparatus']).mean().reset_index()

In [11]:
average_scores

,Name,Apparatus,D_Score,E_Score,Penalty,Score
0,adriel perales-valencia,PB,5.200,8.7000,0.000,13.9000
1,adriel perales-valencia,SR,5.400,7.4500,0.000,12.8500
2,aidan cuy,FX,4.700,8.2000,0.000,12.9000
3,aidan cuy,HB,4.000,7.6500,0.000,11.6500
4,aidan cuy,PB,4.800,8.1500,0.000,12.9500
...,...,...,...,...,...,...
770,zachary nunez,HB,4.500,8.6500,0.000,13.1500
771,zachary nunez,PB,5.125,8.1125,0.000,13.2375
772,zachary nunez,PH,5.650,7.6250,0.000,13.2750
773,zachary nunez,SR,4.700,8.2500,0.000,12.9500


In [13]:
import pandas as pd
import itertools

# Define the dataframe with athlete scores
df = pd.DataFrame({
    'Athlete': ['yul moldauer', 'fred richard', 'vitaliy guimaraes', 'khoi young', 'asher hong'],
    'FX': [14.400, 14.483, 14.250, 13.550, 14.029],
    'PH': [13.957, 13.357, 14.350, 15.342, 12.949],
    'SR': [14.114, 13.350, 13.600, 12.550, 14.566],
    'VT': [14.250, 14.000, 14.450, 15.244, 16.680],
    'PB': [15.121, 14.450, 13.900, 13.750, 14.355],
    'HB': [13.500, 14.962, 13.400, 13.100, 10.450]
})

# Define the number of athletes to select
num_athletes = 4

# Define the apparatuses
apparatuses = ['FX', 'PH', 'SR', 'VT', 'PB', 'HB']

# Generate all possible combinations of athletes
athlete_combinations = list(itertools.combinations(df['Athlete'], num_athletes))

# Initialize variables to store the maximum score and the selected athletes
max_score = 0
selected_athletes = []

# Iterate through each combination of athletes
for combination in athlete_combinations:
    # Generate all possible combinations of apparatuses for the current athlete combination
    apparatus_combinations = list(itertools.combinations(apparatuses, num_athletes))
    
    # Iterate through each combination of apparatuses
    for apparatus_combination in apparatus_combinations:
        # Calculate the sum of the top three scores for the current apparatus combination
        score = df[df['Athlete'].isin(combination)][list(apparatus_combination)].sum(axis=1).nlargest(3).sum()
        
        # Update the maximum score and selected athletes if a higher score is found
        if score > max_score:
            max_score = score
            selected_athletes = list(combination)

# Print the selected athletes and the maximum score
print('Selected Athletes:')
print(selected_athletes)
print('Maximum Score:')
print(max_score)

Selected Athletes:
['yul moldauer', 'fred richard', 'vitaliy guimaraes', 'asher hong']
Maximum Score:
173.798
